In [69]:
# import required libraries
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [70]:
# define function to load data
def load_data(filepath):
    # setting column names of the dataframe
    headers = [
        "X_robot",
        "Y_robot",
        "Orientation_robot",
        "Collision",
        "X_candle1",
        "Y_candle1",
        "X_candle2",
        "Y_candle2",
        "X_candle3",
        "Y_candle3",
        "X_candle4",
        "Y_candle4",
        "X_speed",
        "Y_speed",
    ]

    # load data from file
    df = pd.read_csv(filepath, names=headers)

    return df

In [71]:
# function to calculate Euclidean distance between two points
def euclidean_distance(x1, y1, x2, y2):
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5


# function that performs feature engineering on the dataset
def feature_engineering(robots_df):
    # calculating the distance between the robot and each candle
    robots_df["distance_candle1"] = euclidean_distance(
        robots_df["X_robot"],
        robots_df["Y_robot"],
        robots_df["X_candle1"],
        robots_df["Y_candle1"],
    )
    robots_df["distance_candle2"] = euclidean_distance(
        robots_df["X_robot"],
        robots_df["Y_robot"],
        robots_df["X_candle2"],
        robots_df["Y_candle2"],
    )
    robots_df["distance_candle3"] = euclidean_distance(
        robots_df["X_robot"],
        robots_df["Y_robot"],
        robots_df["X_candle3"],
        robots_df["Y_candle3"],
    )
    robots_df["distance_candle4"] = euclidean_distance(
        robots_df["X_robot"],
        robots_df["Y_robot"],
        robots_df["X_candle4"],
        robots_df["Y_candle4"],
    )

    # calculating the distance between the robot and each candle in each axis
    robots_df["distance_candle1_x"] = robots_df["X_robot"] - robots_df["X_candle1"]
    robots_df["distance_candle1_y"] = robots_df["Y_robot"] - robots_df["Y_candle1"]
    robots_df["distance_candle2_x"] = robots_df["X_robot"] - robots_df["X_candle2"]
    robots_df["distance_candle2_y"] = robots_df["Y_robot"] - robots_df["Y_candle2"]
    robots_df["distance_candle3_x"] = robots_df["X_robot"] - robots_df["X_candle3"]
    robots_df["distance_candle3_y"] = robots_df["Y_robot"] - robots_df["Y_candle3"]
    robots_df["distance_candle4_x"] = robots_df["X_robot"] - robots_df["X_candle4"]
    robots_df["distance_candle4_y"] = robots_df["Y_robot"] - robots_df["Y_candle4"]

    return robots_df

In [72]:
# function to develop and save the model
def develop(X, y):
    # split the data into training, validation and test sets
    X_train_validate, X_test, y_train_validate, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    X_train, X_validate, y_train, y_validate = train_test_split(
        X_train_validate, y_train_validate, test_size=0.125, random_state=42
    )

    # train the model
    hyper_params = {
        "verbose": False,
        "random_state": 33,
        "activation": "relu",
        "hidden_layer_sizes": (12, 24, 48, 96),
        "learning_rate_init": 0.001,
        "batch_size": 50,
    }
    model = MLPRegressor(**hyper_params).fit(X_train, y_train)

    # predict the target on the train, validation and test sets
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_validate)
    y_test_pred = model.predict(X_test)

    # calculate the mean squared error on the train and validation sets
    train_rmse = mean_squared_error(y_train, y_train_pred)
    validate_rmse = mean_squared_error(y_validate, y_val_pred)
    test_rmse = mean_squared_error(y_test, y_test_pred)

    # calculate the mean absolute error on the train, validation and test sets
    train_mae = mean_absolute_error(y_train, y_train_pred)
    validate_mae = mean_absolute_error(y_validate, y_val_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)

    # model props
    model_props = {
        "hidden_layers": model.n_layers_ - 2,
        "architecture": model.hidden_layer_sizes,
        "activation": model.activation,
        "batch_size": model.batch_size,
        "solver": model.solver,
        "learning_rate_init": model.learning_rate_init,
        "learning_rate": model.learning_rate,
        "iterations": model.n_iter_,
        "train_rmse": train_rmse,
        "val_rmse": validate_rmse,
        "test_rmse": test_rmse,
        "train_mae": train_mae,
        "val_mae": validate_mae,
        "test_mae": test_mae,
    }

    # print the model properties
    print(model_props)

    # print the model scores
    print(f"Test scores RMSE: {test_rmse}, MAE: {test_mae}")
    print(f"Validation scores RMSE: {validate_rmse}, MAE: {validate_mae}")
    print(f"Train scores RMSE: {train_rmse}, MAE: {train_mae}")

    # save the model
    with open("./out/model_1.pkl", "wb") as f:
        pickle.dump(model, f)

In [73]:
# function to evaluate the model
def evaluate(X, y):
    model = pickle.load(open("./out/model_1.pkl", "rb"))
    # predict the target on the test set
    y_pred = model.predict(X)

    # evaluate the model
    test_rmse = mean_squared_error(y, y_pred)
    test_mae = mean_absolute_error(y, y_pred)

    # model props
    model_props = {
        "hidden_layers": model.n_layers_ - 2,
        "architecture": model.hidden_layer_sizes,
        "activation": model.activation,
        "batch_size": model.batch_size,
        "solver": model.solver,
        "learning_rate_init": model.learning_rate_init,
        "learning_rate": model.learning_rate,
        "iterations": model.n_iter_,
        "test_score": test_rmse,
        "test_mae": test_mae,
    }

    # print the model properties
    print(model_props)
    # print the model score
    print(f"Test scores  RMSE: {test_rmse}, MAE: {test_mae}")

In [74]:
# the main function that runs the program
def main(mode="develop", file_path="./data/RobotData.csv"):

    # Load data
    df = load_data(file_path)

    # target variables
    target = ["X_speed", "Y_speed"]

    # get feature and target vectors
    X = df.drop(target, axis=1)
    y = df[target]

    # scale the data
    if mode == "develop":
        develop(X, y)
    else:
        evaluate(X, y)

In [75]:
# run the main function
main("develop", "./data/RobotData.csv")

{'hidden_layers': 4, 'architecture': (12, 24, 48, 96), 'activation': 'relu', 'batch_size': 50, 'solver': 'adam', 'learning_rate_init': 0.001, 'learning_rate': 'constant', 'iterations': 19, 'train_rmse': 0.007381068902074671, 'val_rmse': 0.007768264985211542, 'test_rmse': 0.007888354736637352, 'train_mae': 0.07331780018223708, 'val_mae': 0.0753911890724041, 'test_mae': 0.07567950938720489}
Test scores RMSE: 0.007888354736637352, MAE: 0.07567950938720489
Validation scores RMSE: 0.007768264985211542, MAE: 0.0753911890724041
Train scores RMSE: 0.007381068902074671, MAE: 0.07331780018223708


In [76]:
# run main in evaluate mode
main("evaluate", "./data/RobotData.csv")

{'hidden_layers': 4, 'architecture': (12, 24, 48, 96), 'activation': 'relu', 'batch_size': 50, 'solver': 'adam', 'learning_rate_init': 0.001, 'learning_rate': 'constant', 'iterations': 19, 'test_score': 0.007521259696380296, 'test_mae': 0.0739975488871175}
Test scores  RMSE: 0.007521259696380296, MAE: 0.0739975488871175
